## What we have done:  

We downloaded the genetic data set found [here](https://www.ebi.ac.uk/gxa/sc/experiments/E-GEOD-100911/results?colourBy=metadata&metadata=genotype), and then loaded the data set into a dataframe. We then used the [Entrez API](https://www.ncbi.nlm.nih.gov/books/NBK25501/) to convert the gene transcript IDs to KEGG IDs for easier interpretation. As the Entrez API has a low request frequency limit, we collected the gene IDs and saved them in a csv file. With our dataset downloaded and cleaned, we began exploring the dataset more thoroughly. To date we have downloaded and integrated the [UMAP learn](https://umap-learn.readthedocs.io/en/latest/) toolkit into our scikit learn toolkits. We have then begun to cluster the data based on the UMAP-reduced dimensions.

For further exploration, we have also explored the gene ontologies, specifically with the biological function, and have grouped genes associated with similar functions, and have mapped the ontologies to the cells in the genetic dataset. We hope to use these related functions to explore the idea of reducing multiple genes responsible for the same function into a single representative gene.


## Peer feedback:  
#### Bernard Li:  
 - Go into more detail for the UMAP method.
  - UMAP will be used for dimension reduction.
 - Are we going to use different methods?
  - We’ll use UMAP for the reduction to work with the complexity of the data and  use additional methods for clustering.
 - The scope of the data will need a lot of processing to make the data reworkable.
  - We have already set up an annotation dataframe with information on the genes, but we are dropping the information about the pathways and instead focusing on the gene ontology processes for the genes expressed in a cell.
 - Discuss availability of lab data for the ethics section.
  - The lab data is unpublished, so it'll be shared through Google Drive.
 - Are we using the same species for the toy dataset and full dataset?
  - Yes, both the toy dataset and the full dataset are from zebrafish.
 - For the purposes of this project, clarify the scope of the project for the audience or just go in a direction like following one gene for a demonstration.
  - We’ll cluster the cells into cell ‘types’, and then run linear regression to compare the different states, and then display annotations such as function and gene ontology for the top relevant genes.

#### Lam Nguyen:
 - Do we have a clear objective or just the tool?
  - We need to focus a little more on a specific analysis instead of the features of the tool.
 - What are we running linear regression on?
  - We were originally planning on running linear regression between the different cell clusters to compare the gene expression of each differentiation state.
 - Visualization tool drives the analysis, but it's more of an exploration.  Add more analysis to really improve the project.
  - We'll focus on comparing the gene ontology processes between clusters of cells.

## Planned deviations from initial proposal:  
As was suggested by our insightful peer reviewers, Bernard and Lam, we had an ambitious agenda for our project. Nearly halfway through the timeline of our project, we are realizing that we were overly vague in what we plan to accomplish within our time window. As such, we have decided to focus our efforts on creating an interactively-generated histogram of gene ontologies. More specifically, we will reduce our genetic data to two dimensions using UMAP and plot our cells in 2-D. We will cluster the cells into various differentiation steps, and then invite the user to select a region of interest within or across these clusters. Cells within this region of interest will be automatically analyzed and a histogram of the top ten most common gene ontologies will be generated. In practice, this allows the user to select a region of interest and rapidly identify primary shared biological traits between the cells, providing a platform for researchers to quickly understand the biological processes that drive cell differentiation at any point along the differentiation continuum. 
  
With a greater focus on gene ontology, we plan to eliminate several features previously proposed for our tool. First, we will be visualizing the data in two-dimensions. This aids in selecting a region of interest. Second, we will eliminate the ability for the user to interactively choose the parameters for the UMAP reduction. We hope that a more focused purpose for our tool will more than offset the deviations from the original proposal.

In [ ]:
from scipy import io
import pandas as pd
import umap # First time you run this enter pip install umap-learn in your Anaconda Prompt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, scale
from sklearn.decomposition import PCA 
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})

# Obtaining data and data cleaning

In [ ]:
def mtx_to_df (mtf, columns_file, rows_file):
    """Takes and mtf file path, a column file path and a row file path of an mtf file and returns a pandas dataframe"""
    import pandas as pd
    cols = []
    rows = []
    data = io.mmread(mtf)
    with open(columns_file) as file:
        for line in file:
            cols.append(line.rstrip())
    
    with open(rows_file) as file:
        for line in file:
            rows.append(line.rstrip().split('\t')[0])
    arr = data.toarray()
    pd.DataFrame(arr, index = rows, columns = cols).to_csv('norm_counts_data.csv')

In [ ]:
norm = 'E-GEOD-100911.aggregated_filtered_normalised_counts.mtx'
c = 'E-GEOD-100911.aggregated_filtered_normalised_counts.mtx_cols'
r = 'E-GEOD-100911.aggregated_filtered_normalised_counts.mtx_rows'

#mtx_to_df(norm,c,r)
norm_counts = pd.read_csv('norm_counts_data.csv')

In [ ]:
# This was run to convert the gene transcript IDs to KEGG IDs, since this process took so long
# the result was saved to a csv (geneID.csv) file and this code is not needed anymore. 

# from Bio import Entrez
# import time
# import csv

# Entrez.email = "mdpouls1@gmail.com"
# geneIDs = []
# rowsTest = rows[15000:]
# i = 0
# while i < len(rowsTest):
#     for t in range(3):
#         if i < len(rowsTest):
#             handle = Entrez.esearch(db="gene", term=rowsTest[i])
#             record = Entrez.read(handle)
#             if len(record['IdList']) == 1:
#                 geneIDs.append(record['IdList'][0])
#             elif len(record['IdList']) > 1:
#                 geneIDs.append('multiple')
#             else:
#                 geneIDs.append('NaN')
#             handle.close()
#             i += 1
#     time.sleep(1)

# with open('geneID.csv', 'a') as f:
#     for gene in geneIDs:
#         f.write('{},'.format(gene))


In [ ]:
def getAnnotations(uniprotData):
    """Takes the uniprotData file and creates a csv file with the gene ontologys (biological process) for each gene.
    uniprot chart must include a gene ontology (biological process) column and either a Ids column or Gene names column. 
    The created csv has the gene ontologies as  columns and genes as rows with a 0 if the gene does not contain the GO term 
    and a 1 if it does"""
    
    #read the csv into a dataframe
    annotationData = pd.read_table(uniprotData)
    
    #get a list of all the used GO terms in the dataset
    GO = []
    go_dict = {}
    for line in annotationData['Gene ontology (biological process)'].str.split(';'):
        if type(line) == list:
            for item in line:
                GO.append(item.strip())

    #create a dictionary with an empty list for each GO term
    for term in GO:
        if term not in go_dict:
            go_dict[term] = []

    #check each gene 
    for line in annotationData['Gene ontology (biological process)'].str.split(';'):
        item_dict = {}
        if type(line) == list:
            for item in line:
                item_dict[item.strip()] = True
            for k,v in go_dict.items():
                if k in item_dict:
                    v.append(1)
                else:
                    v.append(0)


    go_df = pd.DataFrame.from_dict(go_dict)
    
    if "Ids" in annotationData.columns:
        go_df['geneId'] = annotationData['Ids']
        go_df = go_df.set_index('geneId')
    
    if 'Gene names' in annotationData.columns:
        go_df['geneName'] = annotationData['Gene names']
        go_df = go_df.drop_duplicates(subset = 'geneName')
        go_df = go_df.set_index('geneName')

    go_df.to_csv('goData.csv')
    return(go_df)
    


In [ ]:
def getCellGOTerms(norm_counts,GO_dataframe, filepathName):
    """Takes the normalized counts as a dataframe of normalized counts with cell IDs as the columns and gene names as the rows.
    The normalized counts dataframe must have the column with the gene IDs named '1'. the GO_dataframe parameter is the output 
    from the getAnnotations function. The filepathName is the name of the file you want the csv to be written to
    
    Writes a csv of the relative amounts of a gene ontology term a cell has based off the differentially expressed genes. The
    csv has the GO term as colums and the cell ID as rows."""
    
    norm_counts_id = norm_counts.set_index("1")
    srr_dict = norm_counts_id.to_dict()


    #Convert the go_df datafram into a dictionary of lists for the indices, columns, and data
    gene_dict = GO_dataframe.to_dict('split')

    #create a dictionary with the gene ID as the key and a list of the gene ontologies for that gene
    geneID_dict = {}
    for ind, data in gene_dict.items():
        geneID_dict[ind] = data


    ids = gene_dict['index']
    data = gene_dict['data']

    geneID_dict = {}
    for i in range(len(ids)):
        geneID_dict[ids[i]] = data[i]

    srr_genes_dict_values = {}
    for srr,genes in srr_dict.items():
        genes_GO = np.zeros((len(GO_dataframe.columns),), dtype=int)
        for gene,value in genes.items():
            if value > 0:
                if gene in geneID_dict:
                    genes_GO = np.add(genes_GO,np.multiply(value, np.array(geneID_dict[gene])))
        srr_genes_dict_values[srr] = genes_GO

    #convert to a dataframe
    cols = GO_dataframe.columns
    gene_annotation_values_df = pd.DataFrame.from_dict(srr_genes_dict_values, orient='index', columns = cols)
    #write data to a csv file
    gene_annotation_values_df.to_csv('{}.csv'.format(filepathName))

In [ ]:
# gagnonData_path = "uniprot_gagnon.tab.gz"
# toyData_path = "uniprotData.tab"

# testis = getAnnotations(gagnonData_path)
# kidney = getAnnotations(toyData_path)

# getCellGOTerms(norm_counts,testis,'GeneAnnotation_gagnon')
# getCellGOTerms(norm_counts,kidney,'GeneAnnotation')

In [ ]:
norm_counts = norm_counts.set_index('Unnamed: 0')
norm_transpose = norm_counts.transpose()

# UMAP Reduction:

See the UMAP Documentation page for details. The module import cell at the top of this notebook has instructions for downloading the scikit learn plug-in for UMAP.

NOTE This is a first pass - I'm not super sure it is ready to go. Next steps

1 We need to cluster the data and assign tags to the data points

2 Add a third dimension to the UMAP reduction

3 Plot data in altair to allow interaction

4 Add interaction steps, tags, whatever else we're doing!


In [ ]:
norm_counts_temp = norm_counts
norm_counts_temp = norm_counts_temp.drop(['geneIDs'],axis=1).transpose()
norm_vals = norm_counts_temp.values

In [ ]:
norm_vals_scaled = StandardScaler().fit_transform(norm_vals)

In [ ]:
UMAP_reducer = umap.UMAP()
reduced_genes = UMAP_reducer.fit_transform(norm_vals_scaled)
print('Done')

In [ ]:
plt.scatter(reduced_genes[:,0],reduced_genes[:,1])
plt.gca().set_aspect('equal','datalim')
plt.title('UMAP projection of Zebra Fish genes, unclustered')
plt.show;

In [ ]:
X = norm_vals_scaled
# PCA
pca_mod = PCA(n_components = 7)
data_pca = pca_mod.fit_transform(X)
PCs =pca_mod.components_
PCs.shape

In [ ]:
UMAP_mod = umap.UMAP(n_neighbors = 7, min_dist = 0.2, n_components = 3).fit_transform(X)

In [ ]:
# K means
y_pred = KMeans(n_clusters=7, max_iter=5).fit_predict(X)

In [ ]:
# colors
colors = ListedColormap(sns.color_palette('bright', 7).as_hex())
# plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(UMAP_mod[:, 0], UMAP_mod[:, 1],zs= UMAP_mod[:, 2], c=y_pred, cmap= colors, s=20)
ax.set_xlabel('Dimension 1')
ax.set_ylabel('Dimension 2')
ax.set_zlabel('Dimension 3')
plt.show()

In [ ]:
plt.scatter(UMAP_mod[:,0],UMAP_mod[:,1], c = y_pred, cmap = colors, s=15)
plt.gca().set_aspect('equal','datalim')
plt.title('UMAP projection of Zebra Fish genes, unclustered')
plt.show;

## Get top gene ontologies:  
**The** top gene ontologies are defined in this project as those that have the largest (absolute) parameter value in the final equation for the principal direction. 

In [ ]:
annotation = pd.read_csv('GeneAnnotation.csv')
annot_vals = annotation.values

annot_scaled = StandardScaler().fit_transform(annot_vals)

# PCA on gene ontologies: Use this to figure out het
pca_mod = PCA(n_components = 1) 
data_pca = pca_mod.fit_transform(annot_scaled)
PCs =pca_mod.components_

In [ ]:
PCs_df = pd.DataFrame(PCs)
PCs_df = PCs_df.transpose()
PCs_df[1] = abs(PCs_df[0])
topGo = PCs_df[1].sort_values(ascending = False).head(100).index #List of the index of the top gene ontologies

In [ ]:
top_annotations = annotation.iloc[:,topGo] # Grab the top 100 annotations
labels=top_annotations.columns
min_max_scaler = preprocessing.MinMaxScaler()
top_annotations[:] = min_max_scaler.fit_transform(top_annotations.values) #Scale the data to be in range [0,1]
top_annotations.columns = np.arange(0,len(top_annotations.columns))
top_annotations.head() #df with only the top 100 most important group ontologies listed for each cell

## Visualization:
**Overview**
 - The 'base' plot is the 2-D UMAP plot plotted using Altair. 
 - A user can then select a region-of-interest to create the breakout histogram plots of predominate gene ontology

In [ ]:
import altair as alt

### Create visualization dataframe:  
**Overview** 

In [ ]:
# Create 'base' dataframe:
column_names = ["UMAP x","UMAP y"]
vis_data = pd.DataFrame(reduced_genes, columns = column_names)
vis_data["Clusters"] = y_pred # Add clusters:
vis_data['IDs'] = top_annotations.index.values
# vis_data[top_annotations.columns] = top_annotations.values # Bring in the 'top annotations'

# vis_data.head()
# vis_data.plot.bar(y=1)
vis_data

In [ ]:
## CHOOSE NUMBER OF ONTOLOGIES TO BE IN CHART!!!
num_ontologies = 20

# Ceate a 'long' chart of the form [CellID | Annotation type | Annotation Value] (and probably UMAPs...)
annotation_play = top_annotations
annotation_play['ids'] = annotation_play.index
vals = np.arange(0,num_ontologies)
annotation_play = pd.melt(top_annotations, id_vars=['ids'], value_vars=vals, var_name = 'ontology')
annotation_play['UMAP x'] = ''
annotation_play['UMAP y'] = ''
annotation_play['Cluster'] = ''

## ADD THE UMAP LOCATIONS AND CLUSTER ID FOR EACH INSTANCE OF EACH CELL:
# PS: Kinda hacked this one together.... so it takes a quick sec to run
cnt = 1
for i in range(len(annotation_play)):
    for j in range(len(vis_data)):
        if annotation_play.loc[i, 'ids'] == vis_data.loc[j,'IDs']:
            annotation_play.loc[i, 'UMAP x'] = float(vis_data.loc[j,'UMAP x'])
            annotation_play.loc[i, 'UMAP y'] = float(vis_data.loc[j,'UMAP y'])
            annotation_play.loc[i, 'Cluster'] = int(vis_data.loc[j,'Clusters'])
#             print(len(annotation_play) - cnt)
            cnt += 1

annotation_play

### Create 'master' plot  
**TODO** 
 - Add cluster colors
 - Add IDs when hovering?
 - Add ROI selection


### Create histogram plot of ontologies:
**Thoughts:**
 - Altair can plot more than the 'maxrows' limit, although it becomes very sluggish. They recommend to save the data locally as JSON and reference a path to it.

In [ ]:
# Create labels dataframe:
labels.tolist()
df_labels = pd.DataFrame(labels.tolist(), columns=['labels'])
df_labels = df_labels.loc[:num_ontologies,:]
df_labels

### Display tool!
**This is where our interaction is!**

In [ ]:
# Save data to a local JSON format:
data_JSON = alt.data_transformers.enable('json')
# Turn off max_rows limit -- this makes the visualization clunky
alt.data_transformers.disable_max_rows()

ROI = alt.selection_interval()
click = alt.selection_multi()

scatter_base = alt.Chart(annotation_play).mark_circle().encode(
    x='UMAP x',
    y='UMAP y',
    color = alt.Color('Cluster:N', scale=alt.Scale(scheme = 'set1'))
).properties(
    width = 500,
    height = 200
).add_selection(ROI, click)

br_sz = 20
hist_base = alt.Chart(annotation_play).mark_bar(size=br_sz).encode(
    x='ontology',
    y='mean(value):Q'
).properties(
    width = 500,
    height = 200
)

hist_roi = alt.Chart(annotation_play).mark_bar(size=br_sz, opacity=0.5).encode(
    alt.X('ontology'),
    alt.Y('mean(value):Q',
          scale=alt.Scale(domain=(0,1))),
    color=alt.value('red')
).properties(
    width = 500,
    height = 200
).transform_filter(
    ROI | click
)

# hist_labels = alt.Chart(df_labels).mark_text(align='center', baseline='bottom',
#                                  dy=35, fontSize=12
#                                  ).encode(
# y=alt.value(100),
# text='labels:N')


scatter_base & (hist_base + hist_roi)
# hist_labels

In [ ]:
# Show ontology labels for histogram above
for i, ont in enumerate(df_labels['labels']):
    print(i, ':',  ont)

## Main Dataset

 This is unpublished data from the Gagnon lab from the [cell ranger](https://support.10xgenomics.com/single-cell-gene-expression/software/pipelines/latest/what-is-cell-ranger) pipeline. I loaded the files, cleaned the data and created a (large) csv file to load in.  As discussed in peer feedback, the unpublished data will be shared will a Google Drive link below.

In [ ]:
#testis1_data_ = io.mmread('matrix.mtx.gz')
#data_arr = testis1_data_.toarray()
#data_col = pd.read_csv('barcodes.tsv.gz', compression = 'gzip', header=None, sep = '\t')
#data_row = pd.read_csv('features.tsv.gz', compression = 'gzip', header=None,sep = '\t')
#data_rows = data_row[1]
#data_cols = []
#for b in data_col[0]:
#    data_cols.append(b)
#data_counts = pd.DataFrame(data_arr, index = data_rows, columns = data_cols)
#data_counts.to_csv('data_counts_data.csv')

### Here is the link to the csv file I generated with the code above: [t1_counts_data.csv](https://drive.google.com/file/d/1aIR4w9TIOnMxziE5aQVjIMAWvqEY6Mvq/view?usp=sharing)

In [ ]:
# The csv file above needs to be downloaded to load this cell
# It is a large dataset, so it may take a while to load
data_counts = pd.read_csv('t1_counts_data.csv', header = 0)
data_counts.set_index('1', inplace = True)
X = data_counts
data_counts.shape # original size

# Quality control of the cells

In [ ]:
# percentage of mitochondrial rna
mitochondrial = data_counts[data_counts.index.str.contains('mt-') > 0].sum(0)
total = data_counts.sum(0)
mito_percentage = (mitochondrial/total) * 100
filter_out = mito_percentage[mito_percentage < 5].index
data_counts = data_counts[filter_out] # filter out anything higher than 5%

# filter by counts of genes (a very large amount of transcribed genes implies more than one cell is captured)
gene_counts = []
for i,cell in enumerate(data_counts.columns):
    counter = 0
    for row in data_counts.iloc[:,i]:
        if row > 0:
            counter += 1
    gene_counts.append(counter)
count_mask = []
for count in gene_counts:
    if count > 200 & count < 2500:
        count_mask.append(gene_counts.index(count))
data_counts = data_counts.iloc[:,count_mask]
data_counts.shape # cutting down cells check

In [ ]:
# Define features (genes) and samples (cell_ids)
genes = data_counts.index.values
cell_ids = data_counts.columns

In [ ]:
indicies = np.arange(0,8627)
mask = np.ones((len(indicies),1))
for i in range(len(mask)):
    if indicies[i] not in count_mask:
        mask[i] = 0
        #print('count')
    elif indicies[i] not in filter_out:
        mask[i] = 0
        #print('indicies')
sum(mask==0)

In [ ]:
# Scale data to get mean-variance standardized values
X = data_counts.to_numpy()
ss_mod = StandardScaler()
X = ss_mod.fit_transform(X)
# ^ these values are just mean-variance standarized, use the variance for each gene and filter out highest (2,000 is recommended)

In [ ]:
# Find variances of each gene and sort for top variable genes
gene_variances = np.var(X, axis=1)
top_variable_genes = pd.DataFrame(gene_variances, index=genes, columns=['variances'])
top_variable_genes['inds'] = [i for i in range(0,32520)]
tvg = top_variable_genes.sort_values('variances', ascending=False)[:2000] # Top Variable Genes

### Subsetting

In [ ]:
### Removing duplicates by index comparisons (gene name leads to doubles) and subsetting original data_counts
tvg_inds = tvg['inds']
tvgi = tvg.index.values
tvgidf = pd.DataFrame(tvgi)
# save csv
# tvgidf.to_csv('top_genes.csv')

In [ ]:
# Manually troubleshooting gene names
tdc = data_counts[data_counts.index.isin(tvgidf.iloc[:,0])].drop_duplicates() #top data_counts with duplicates
tdci = tdc.index.values # #top data_counts genes
tdci.sort() # Alphabetically orders both so that they can be compared 
tvgi.sort() # to the top variable genes list
tdcil = tdci.tolist()
# troubleshooting dataset doubles, rm holds the dropped indexs for the duplicates
rm = [122,213, 250, 298, 437, 561,637, 700, 701, 706, 730, 976, 1339, 1620, 1685, 1766, 1846, 1972, 1981]
# uncomment prints to see duplicates
for i in rm:
    if tvgi[i] == tdci[i]:
            print('Clear')
    elif i != 701:
            #print(i-1, tvgi[i-1], tdci[i-1])
           # print('===',i,tvgi[i], tdci[i])
           # print(i+1,tvgi[i+1], tdci[i+1])
            tdcil.remove(tdci[i])
            tdci = np.asarray(tdcil)
           # print('AFTER')
           # print(i-1, tvgi[i-1], tdci[i-1])
           # print('===',i,tvgi[i], tdci[i])
           # print(i+1,tvgi[i+1], tdci[i+1])
            #print()
    elif i == 701:
           # print(i-1, tvgi[i-1], tdci[i-1])
           # print('===',i,tvgi[i], tdci[i])
           # print(i+1,tvgi[i+1], tdci[i+1])
            tdcil.remove(tdci[i-1]) # shift index
            tdci = np.asarray(tdcil)
            #print('AFTER:')
            #print(i-1, tvgi[i-1], tdci[i-1])
            # print('===',i,tvgi[i], tdci[i])
            #print(i+1,tvgi[i+1], tdci[i+1])
            #print()
print(len(tdci))

# checking datasets
for i in range(0,2000):
    if tvgi[i] != tdci[i]: # they are the same if the ERROR isn't printed
        print('ERROR:', i, tvgi[i], tdci[i])

#### From the highest variable genes, there were multiple genes that had more than one row in data_counts, so: this section puts the highest expression (closer to a large variability score) in and adding to the list of rows to subset from the original 32,520 genes:

In [ ]:
###### CAN ONLY RUN ONCE
dc_inds = [] # original indicies from data_counts to reference
indices = [] # Subsetting mask
dupls = [] # list of duplicates that aren't filtered out by dropping duplicates (.drop_duplicates() can't catch these because of genes with doubles)

# add the indicies of genes in the original set
for g in tdci: 
    dc_inds.append(top_variable_genes.loc[g, 'inds'])
for i in dc_inds:
    if type(i) == np.float64:
        indices.append(int(i))
    else:
        dupls.append(i)

# for the duplicates compare the rows of duplicate data_counts (which are different), find the set with the higher expression
for dup in dupls:
    if dup.shape[0] == 2:
        a = dup.iloc[0]
        b = dup.iloc[1]
        Aa = data_counts.iloc[a, :]
        Ab = data_counts.iloc[b, :]
        AA = Aa.compare(Ab)
        asum = AA['self'].sum()
        bsum = AA['other'].sum()
        if asum > bsum:
            if a not in indices:
                indices.append(a)
        elif b not in indices:
            indices.append(b)
    else: # this is for the triple duplicates which compares all three
        a = dup.iloc[0]
        b = dup.iloc[1]
        Aa = data_counts.iloc[a, :]
        Ab = data_counts.iloc[b, :]
        AA = Aa.compare(Ab)
        asum = AA['self'].sum()
        bsum = AA['other'].sum()
        if asum > bsum:
            w = a
            Ww = Aa
            winsum = asum
        else:
            w = b
            Ww = Ab
            winsum = bsum
        c = dup.iloc[2]
        Ac = data_counts.iloc[c, :]
        AB = Ww.compare(Ac)
        csum = AB['other'].sum()
        if csum > winsum:
            if c not in indices:
                indices.append(c)
        elif w not in indices:
            indices.append(w)
indices.sort() # sort by order of indicies
print(len(indices)) # check shape before

## Finally, subset with the indicies mask

In [ ]:
data_counts = data_counts.iloc[indices, :] 
print(data_counts.shape) # check shape after to compare from before subsetting,
# now it should show 1998 (2 of the duplicates had triples)

# save csv
#data_counts.to_csv('data_counts.csv')

# Reset X from the original shape to the top variable genes subset
X = data_counts.to_numpy()
X.shape
# Redefine features and samples
genes = data_counts.index.values
cell_ids = data_counts.columns

### Generate descriptions for the top variable genes

In [ ]:
# Descriptions of genes  
genes_desc = pd.read_csv('GENE-DESCRIPTION-TXT_ZFIN_17.txt', names=['gene'], skiprows=20, sep='\t') #downloaded from: https://www.alliancegenome.org/downloads#gene-descriptions
# gene descriptions (the _d descriptions and the _g genes)
gd_d = genes_desc.index.values.tolist()
gd_d = gd_d[1::2]
gd_g = genes_desc.iloc[:,0].dropna().tolist()

genes_descriptions = pd.DataFrame(gd_d, index = gd_g, columns = ['Description'])
print(genes_descriptions.shape)

gene_mask = genes.tolist()
gm_inds = []
for i,g in enumerate(genes_descriptions.index):
    if g in gene_mask:
        gm_inds.append(i)
tvg_gd = genes_descriptions.iloc[gm_inds, :] # Subsetted gene descriptions
tvg_gd.shape # only found descriptions for 1852

In [ ]:
gnf_g = []
for g in genes:
    if g not in tvg_gd.index.values:
        tvg_gd.loc[f'{g}'] = 'Gene not found'
tvg_gd.shape

In [ ]:
tvg_gd.head()

In [ ]:
top_genes = pd.read_csv('top_genes.csv')
dc2 = data_counts.transpose()
dc2[top_genes['0'].drop_duplicates()].head()

### Apply the same UMAP pipeline as above with new data:

**Pipeline for finding neighbors:**
 - Set test and train datasets
 - Set a list of k's like [k for k in range(2,10)] or something like that
 - Build loops for k's that sets the k-nn classifier as knn_model, then use knn_model.fit for the data, set y_pred with     knn_model.predict(X_test) and use the metrics.accuracy_score(y_true=y_test, y_pred=y_pred) for scores for each k

In [ ]:
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors

In [ ]:
norm_vals = data_counts.transpose().values
# norm_vals = dc2.values
norm_vals_scaled = StandardScaler().fit_transform(norm_vals)
print('Done')

In [ ]:
# Split data for development purposes:
X_train, X_test = train_test_split(norm_vals_scaled, test_size=0.75, random_state=1) #Reduce the number of cells for the clustering:
print('done')
# UMAP_train = umap.UMAP(n_neighbors = 7, min_dist = 0.2, n_components = 2).fit_transform(X_train)

## Optimizing the UMAP parameters:
**We** manually ran the two boxes below with varying parameters and visually assessed the results to identify our 'best' parameters for the UMAP reduction

In [ ]:
'''UMAP_mod = umap.UMAP(n_neighbors = 13, min_dist = .5, n_components = 2).fit_transform(X_train)
print('done')'''

In [ ]:
'''plt.scatter(UMAP_mod[:,0],UMAP_mod[:,1])
plt.gca().set_aspect('equal','datalim')
plt.title('UMAP projection of Zebra Fish genes, unclustered')
plt.show;'''

In [ ]:
def np_to_csv(numpy_array, write_name):
    df_temp = pd.DataFrame(numpy_array)
    df_temp.to_csv(write_name)
    print('Successfully wrote array as ' + write_name)

## Optimize kmeans clustering!

In [ ]:
'''n_rng = np.arange(3,9)
score = []
for n in n_rng:
    print('Working on: %d' % n)
    mdl = KMeans(n_clusters = 12, max_iter=10, random_state=1, ).fit(X_train)
    score.append(metrics.silhouette_score(x_train, mdl.labels_, metric='euclidean'))
print(n_rng)
print(score)''''''plt.scatter(UMAP_mod[:,0],UMAP_mod[:,1])
plt.gca().set_aspect('equal','datalim')
plt.title('UMAP projection of Zebra Fish genes, unclustered')
plt.show;'''

### Run the UMAP with the final parameters!

In [ ]:
UMAP_mod = umap.UMAP(n_neighbors = 27, min_dist = 0.70, n_components = 2, random_state=10).fit_transform(norm_vals_scaled) #Double the number of neighbors bec. we double sz of data
# ^ UNCOMMENT TO RUN! IT TAKES FOREVER

plt.scatter(UMAP_mod[:,0],UMAP_mod[:,1])
plt.gca().set_aspect('equal','datalim')
plt.title('UMAP projection of Zebra Fish genes, unclustered')
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')
plt.show;

np_to_csv(UMAP_mod, 'Gagnon_UMAP.csv')

### Finding optimal k

In [ ]:
ks = range(1,21)
scores = []
for k in ks:
    model = KMeans(n_clusters=k)
    y_pred = model.fit_predict(X)
    scores.append(-model.score(X))
fig = plt.figure(figsize=(14, 8))
plt.plot(ks, scores)
plt.ylabel('total intra-cluster distance')
plt.xticks([i for i in range(1,20)])
plt.xlabel('k')
plt.show()

## Run the final clustering!

In [ ]:
nc = 12
y_pred = KMeans(n_clusters=nc, max_iter=5).fit_predict(norm_vals_scaled)# UNCOMMENT IF YOU WANT TO RUN! IT TAKES FOREVER!
np_to_csv(y_pred, 'Gagnon_kMeans.csv') # Save the prediction

## Show the final results of the UMAP reduction with clustering!

In [ ]:
colors = ListedColormap(sns.color_palette('Paired', nc).as_hex())
df_temp = pd.read_csv('Gagnon_UMAP.csv')
UMAP_mod = df_temp[['0','1']].values
scatter = plt.scatter(UMAP_mod[:,0],UMAP_mod[:,1], c = y_pred, cmap = colors, s=14)
plt.gca().set_aspect('equal','datalim')

plt.title('UMAP projection of Zebra Fish genes, unclustered')
plt.xlabel('UMAP 1')
plt.ylabel('UMAP 2')
plt.legend(*scatter.legend_elements())
plt.show;

In [ ]:
# make empty containers with clusters
def make_cl_cont(cs, listfill=False):
    cl_dict = {}
    for cl in cs:
        if listfill:
            cl_dict[cl] = []
        else:
            cl_dict[cl] = 0
    return cl_dict

In [ ]:
# Subset clusters
cs = [f'cluster{i}' for i in range(0,nc)]
cl_subs = make_cl_cont(cs, listfill=True)
for i,cc in enumerate(y_pred):
    cl_subs[f'cluster{cc}'].append(i)
len(cl_subs)
# generate sum for each gene for each cell in each cluster
g_cls = make_cl_cont(cs)
for cl in cl_subs:
    cl_g_sums = []
    cl_c = data_counts.iloc[:,cl_subs[cl]]
    for row in cl_c.index:
        cl_g_sums.append(cl_c.loc[row].sum())
    g_cls[cl] = cl_g_sums
tg_cls =make_cl_cont(cs)
for cl in g_cls:
    gene_df = pd.DataFrame(g_cls[cl], index=genes, columns=['cl_sums'])
    gene_df = gene_df.sort_values('cl_sums', ascending=False) # sort from highest to lowest
    tg_cls[cl] = gene_df.index.values[:25] # select top 25
len(tg_cls)
hm_inds = make_cl_cont(cs,listfill=True)
for cl in tg_cls:
    for g in tg_cls[cl]:
        for i,row in enumerate(data_counts.index):
            if row == g:
                hm_inds[cl].append(i)

In [ ]:
# Visualize Cluster and top expressed genes by input (run for each cluster)
cl_v = input('Cluster number: ')
plt.title(f'Cluster {cl_v} Top Expressed Genes')
plt.xlabel('Cells')
plt.ylabel('Genes')
plt.tick_params(left=True, bottom=False)
sns.heatmap(data_counts.iloc[hm_inds[f'cluster{cl_v}'], cl_subs[f'cluster{cl_v}']]);
with pd.option_context('display.max_colwidth', None):
    display(tvg_gd.loc[tg_cls[f'cluster{cl_v}']])

### Re-create the visualization pipeline:

##### Add annotations:

In [ ]:
annotation = pd.read_csv('GeneAnnotation_gagnon.csv')
annotation.rename(columns={'Unnamed: 0':'index'}, inplace=True)
annotation.set_index('index',inplace=True)
annot_vals = annotation.values
annot_scaled = StandardScaler().fit_transform(annot_vals)

# PCA on gene ontologies: Use this to figure out het
pca_mod = PCA(n_components = 1) 
data_pca = pca_mod.fit_transform(annot_scaled)
PCs =pca_mod.components_
print('done')

In [ ]:
annotation.shape

In [ ]:
PCs_df = pd.DataFrame(PCs)
PCs_df = PCs_df.transpose()
PCs_df[1] = abs(PCs_df[0])
topGo = PCs_df[1].sort_values(ascending = False).head(100).index #List of the index of the top gene ontologies
print('done')

In [ ]:
top_annotations = annotation.iloc[:,topGo] # Grab the top 100 annotations
labels=top_annotations.columns
min_max_scaler = preprocessing.MinMaxScaler()
top_annotations[:] = min_max_scaler.fit_transform(top_annotations.values) #Scale the data to be in range [0,1]
top_annotations.columns = np.arange(0,len(top_annotations.columns))
top_annotations.head() #df with only the top 100 most important group ontologies listed for each cell

#### Final viz dataframe:

In [ ]:
# Create 'base' dataframe:
column_names = ["UMAP x","UMAP y"]
vis_data = pd.DataFrame(UMAP_mod, columns = column_names)
vis_data["Clusters"] = y_pred # Add clusters:

vis_data['IDs'] = top_annotations.index.values
# vis_data[top_annotations.columns] = top_annotations.values # Bring in the 'top annotations'

# vis_data.head()
# vis_data.plot.bar(y=1)
vis_data

**Format the data into a 'long-format' dataframe appropriate for altair**

In [ ]:
## CHOOSE NUMBER OF ONTOLOGIES TO BE IN CHART!!!
num_ontologies = 10

# Ceate a 'long' chart of the form [CellID | Annotation type | Annotation Value] (and probably UMAPs...)
annotation_play = top_annotations
annotation_play['ids'] = annotation_play.index
vals = np.arange(0,num_ontologies)
annotation_play = pd.melt(top_annotations, id_vars=['ids'], value_vars=vals, var_name = 'ontology')
annotation_play['UMAP x'] = ''
annotation_play['UMAP y'] = ''
annotation_play['Cluster'] = ''

## ADD THE UMAP LOCATIONS AND CLUSTER ID FOR EACH INSTANCE OF EACH CELL:
# PS: Kinda hacked this one together.... so it takes a quick sec to run
cnt = 1
for i in range(len(annotation_play)):
    for j in range(len(vis_data)):
        if annotation_play.loc[i, 'ids'] == vis_data.loc[j,'IDs']:
            annotation_play.loc[i, 'UMAP x'] = float(vis_data.loc[j,'UMAP x'])
            annotation_play.loc[i, 'UMAP y'] = float(vis_data.loc[j,'UMAP y'])
            annotation_play.loc[i, 'Cluster'] = int(vis_data.loc[j,'Clusters'])
#             print(len(annotation_play) - cnt)
            cnt += 1

annotation_play

In [ ]:
# Save data to a local JSON format:
data_JSON = alt.data_transformers.enable('json')
# Turn off max_rows limit -- this makes the visualization clunky
alt.data_transformers.disable_max_rows()

ROI = alt.selection_interval()
click = alt.selection_multi()

scatter_base = alt.Chart(annotation_play).mark_circle().encode(
    x='UMAP x',
    y='UMAP y',
    color = alt.Color('Cluster:N', scale=alt.Scale(scheme = 'set1'))
).properties(
    width = 500,
    height = 200
).add_selection(ROI, click)

br_sz = 20
hist_base = alt.Chart(annotation_play).mark_bar(size=br_sz).encode(
    x='ontology',
    y='mean(value):Q'
).properties(
    width = 500,
    height = 200
)

hist_roi = alt.Chart(annotation_play).mark_bar(size=br_sz, opacity=0.5).encode(
    alt.X('ontology'),
    alt.Y('mean(value):Q',
          scale=alt.Scale(domain=(0,1))),
    color=alt.value('red')
).properties(
    width = 500,
    height = 200
).transform_filter(
    ROI | click
)

# hist_labels = alt.Chart(df_labels).mark_text(align='center', baseline='bottom',
#                                  dy=35, fontSize=12
#                                  ).encode(
# y=alt.value(100),
# text='labels:N')


scatter_base & (hist_base + hist_roi)